In [3]:
import pandas as pd
import numpy as n

application_train_df = pd.read_csv('rory_work/application_train.csv')
bureau_df = pd.read_csv('Datasets/bureau.csv')
prev_app_df = pd.read_csv('Datasets/previous_application.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/bureau.csv'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer


application_train_df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)


for col in application_train_df.columns:
    application_train_df[col + "_MISSING"] = application_train_df[col].isna().astype(int)

categorical_cols = application_train_df.select_dtypes(include=["object"]).columns

numeric_like_cols = application_train_df.columns.difference(categorical_cols)

application_train_df[numeric_like_cols] = application_train_df[numeric_like_cols].apply(
    lambda col: pd.to_numeric(col, errors="coerce")
)


numeric_cols = application_train_df.select_dtypes(include=[np.number]).columns
numeric_cols = numeric_cols.drop("TARGET")  


num_imputer = SimpleImputer(strategy="median")
application_train_df[numeric_cols] = num_imputer.fit_transform(application_train_df[numeric_cols])

cat_imputer = SimpleImputer(strategy="most_frequent")
application_train_df[categorical_cols] = cat_imputer.fit_transform(application_train_df[categorical_cols])

# Confirm no NaNs
print("Remaining NaNs:", application_train_df.isna().sum().sum())


df_encoded = pd.get_dummies(application_train_df, drop_first=True)


correlations = df_encoded.corr()["TARGET"]
low_corr_cols = correlations[abs(correlations) < 0.01].index.tolist()


low_corr_cols = [col for col in low_corr_cols if col != "TARGET"]

df_filtered = df_encoded.drop(columns=low_corr_cols, errors='ignore')

print("Final shape:", df_filtered.shape)


X = df_filtered.drop("TARGET", axis=1)
y = df_filtered["TARGET"]


C:\Users\admin\AppData\Local\Temp\ipykernel_17572\255203758.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  application_train_df["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_17572\255203758.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

Remaining NaNs: 0


In [ ]:
# Feature Engineering
application_bureau_df = pd.merge(application_train_df, bureau_df, on = "SK_ID_CURR")
total_overdue = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_OVERDUE"].sum()
total_debt = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_DEBT"].sum()
times_prolonged = application_bureau_df.groupby("SK_ID_CURR")["CNT_CREDIT_PROLONG"].sum()
days_overdue = application_bureau_df.groupby("SK_ID_CURR")["CREDIT_DAY_OVERDUE"].sum()

In [ ]:
application_train_merged_df = application_train_df.merge(total_overdue, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(total_debt, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(times_prolonged, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(days_overdue, on = "SK_ID_CURR", how = 'left')

In [ ]:
application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'] = application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
application_train_merged_df['AMT_CREDIT_SUM_DEBT'] = application_train_merged_df['AMT_CREDIT_SUM_DEBT'].fillna(0)
application_train_merged_df['CNT_CREDIT_PROLONG'] = application_train_merged_df['CNT_CREDIT_PROLONG'].fillna(0)
application_train_merged_df['CREDIT_DAY_OVERDUE'] = application_train_merged_df['CREDIT_DAY_OVERDUE'].fillna(0)

In [ ]:
prev_app_ct = prev_app_df[["SK_ID_CURR"]]
prev_app_ct["PREV_APPS"] = 0
prev_app_ct = prev_app_ct.groupby("SK_ID_CURR").count()
prev_app_ct

C:\Users\nerdc\AppData\Local\Temp\ipykernel_39080\1380318554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_app_ct["PREV_APPS"] = 0


,PREV_APPS
SK_ID_CURR,
100001,1
100002,1
100003,3
100004,1
100005,2
...,...
456251,1
456252,1
456253,2


In [ ]:
prev_app_approved = prev_app_df[["SK_ID_CURR", "NAME_CONTRACT_STATUS"]]
prev_app_approved["NUM_APPROVED"] = np.where(prev_app_approved["NAME_CONTRACT_STATUS"] == "Approved", 1, 0)
prev_app_approved = prev_app_approved.groupby("SK_ID_CURR").sum().reset_index()
prev_app_approved = prev_app_approved[["SK_ID_CURR", "NUM_APPROVED"]]

C:\Users\nerdc\AppData\Local\Temp\ipykernel_39080\326452719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_app_approved["NUM_APPROVED"] = np.where(prev_app_approved["NAME_CONTRACT_STATUS"] == "Approved", 1, 0)


In [ ]:
prev_app_ct = prev_app_ct.merge(prev_app_approved, on="SK_ID_CURR", how="inner")

In [ ]:
application_train_merged_df = application_train_merged_df.merge(prev_app_ct, on="SK_ID_CURR", how="left")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_to_standardize = ['AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG', 'CREDIT_DAY_OVERDUE', 'PREV_APPS', 'NUM_APPROVED']

application_train_merged_df[cols_to_standardize] = scaler.fit_transform(application_train_merged_df[cols_to_standardize])

In [ ]:
application_train_merged_df.shape[1]

129

In [ ]:
# Variable Selection
pd.set_option('display.max_rows', 1000)  

correlation_matrix = application_train_merged_df.corr()
target_column_name = 'TARGET'
target_correlations = correlation_matrix[target_column_name]
print(f"\nCorrelations with '{target_column_name}':")
# print(target_correlations)


sorted_target_correlations = target_correlations.sort_values(ascending=False)
print(f"\nSorted correlations with '{target_column_name}':")
print(sorted_target_correlations)

print(application_train_merged_df['TARGET'])

ValueError: could not convert string to float: 'Cash loans'

In [ ]:
##Columns with less that 0.01 correlation to target

low_correlation_columns = sorted_target_correlations[abs(sorted_target_correlations) < 0.01].index.tolist()
print(f"\nColumns with less than 0.01 correlation to '{target_column_name}':")
print(low_correlation_columns)

application_train_merged_df = application_train_merged_df.dropna()

df_correlation = application_train_merged_df.drop(columns=low_correlation_columns)

df_correlation = df_correlatio.f['TARGET'])


Columns with less than 0.01 correlation to 'TARGET':
['CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'REG_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'FLAG_DOCUMENT_2', 'NAME_FAMILY_STATUS', 'CREDIT_DAY_OVERDUE', 'FLAG_DOCUMENT_21', 'LIVE_REGION_NOT_WORK_REGION', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'FLAG_MOBIL', 'FONDKAPREMONT_MODE', 'FLAG_CONT_MOBILE', 'FLAG_DOCUMENT_20', 'WEEKDAY_APPR_PROCESS_START', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_7', 'NONLIVINGAPARTMENTS_MODE', 'FLAG_EMAIL', 'AMT_REQ_CREDIT_BUREAU_QRT', 'SK_ID_CURR', 'FLAG_DOCUMENT_4', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG', 'FLAG_DOCUMENT_17', 'AMT_INCOME_TOTAL', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_9', 'NAME_TYPE_SUITE', 'FLAG_OWN_REALTY', 'FLAG_DOCUMENT_15', 'OCCUPATION_TYPE', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_8', 'YEA

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM_DEBT,CNT_CREDIT_PROLONG,CREDIT_DAY_OVERDUE,PREV_APPS,NUM_APPROVED
count,6206.000000,6206.000000,6206.000000,6206.000000,6206.0,6206.000000,6206.000000,6.206000e+03,6.206000e+03,6206.000000,...,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000,6206.000000
mean,277628.523526,0.064776,0.103287,0.495972,1.0,0.701096,0.606671,2.196898e+05,6.868752e+05,31460.237432,...,0.041412,0.371092,0.269256,1.843539,0.005001,0.432140,0.092621,-0.025131,0.030644,0.039844
std,103598.458034,0.246150,0.304358,0.500024,0.0,0.457815,0.773960,1.366682e+05,4.460252e+05,16008.211503,...,0.214073,1.049797,0.609232,1.754519,0.530368,1.480172,1.269631,0.592071,0.976472,1.006781
min,100083.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,3.375000e+04,4.500000e+04,3411.000000,...,0.000000,0.000000,0.000000,0.000000,-0.012514,-0.681064,-0.151294,-0.049444,-0.930093,-1.432948
25%,187172.750000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.350000e+05,3.141000e+05,19803.375000,...,0.000000,0.000000,0.000000,1.000000,-0.012514,-0.348799,-0.151294,-0.049444,-0.688957,-0.491587
50%,276788.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,1.890000e+05,5.772802e+05,29299.500000,...,0.000000,0.000000,0.000000,1.000000,-0.012514,-0.093099,-0.151294,-0.049444,-0.206684,-0.020906
75%,369441.250000,0.000000,0.000000,1.000000,1.0,1.000000,1.000000,2.700000e+05,9.423000e+05,40320.000000,...,0.000000,0.000000,0.000000,3.000000,-0.012514,0.564151,-0.151294,-0.049444,0.516724,0.449774
max,456226.000000,1.000000,1.000000,1.000000,1.0,1.000000,5.000000,4.500000e+06,4.050000e+06,171040.500000,...,4.000000,16.000000,8.000000,17.000000,27.509062,28.887978,26.022864,30.807434,11.367850,7.980660


Split code:

In [ ]:
application_train_merged_df.shape[1]

128